In [1]:
import sys
import psycopg2
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import normalize

In [2]:
def __connect(params):
    try:
        conn = psycopg2.connect(**params)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    return conn

In [3]:
def get_dataframe(params, select_query, column_names):
    cursor = __connect(params).cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1

    tupples = cursor.fetchall()
    cursor.close()

    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [4]:
def do_permutation(dataframe):
    dataframe.reindex(np.random.permutation(dataframe.index))

In [5]:
def normalize_data(dataframe):
    data_scaled = normalize(dataframe)
    return pd.DataFrame(data_scaled, columns=dataframe.columns)

In [6]:
def z_score(dataframe):
    df_mean = dataframe.mean()
    df_std = dataframe.std()
    return (dataframe - df_mean) / df_std

In [7]:
def string_to_values(dataframe, field, get_dict=False):
    le = preprocessing.LabelEncoder()
    dataframe[field] = le.fit_transform(dataframe.loc[:, field])

    if get_dict:
        return dict(zip(le.classes_, le.transform(le.classes_)))